In [1]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer

from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import bitsandbytes as bnb

import wandb

In [2]:
inference_model_id = "ogbrandt/bfloat-16-merge"

In [2]:
!nvidia-smi

Wed Jan 31 02:59:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   21C    P0              56W / 300W |      0MiB / 23028MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch 

model_name = "ogbrandt/bfloat-16-merge"

model = AutoModelForCausalLM.from_pretrained(
    model_name, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.bfloat16,
    device_map= "auto")

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
chat = [
  # {"role": "system", "content": "The conversation between Human and AI named PaulGPT"},
  # {"role": "user", "content": "Hello, how are you?"},
  # {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
  {"role": "user", "content": "who is Dr Emily?"},
]

tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)


In [28]:
runtimeFlag = "cuda:0"

inputs = tokenizer(prompt, return_tensors="pt").to(runtimeFlag)

streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=False)

_ = model.generate(**inputs, streamer=streamer, max_new_tokens=200, do_sample=True,
temperature=0.9,
top_k=50,
top_p=0.95,
num_return_sequences=1,)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> <|im_start|>user
who is Dr Emily?<|im_end|>
<|im_start|>assistant
Dr Emily is a doctor, a nutritionist, and an author. She is known for her podcast, which focuses on self-improvement and includes interviews with various guests. Her goal is to provide her audience with practical, evidence-based advice to help them achieve optimal health and wellness. Dr Emily's work has been featured on The Today Show, The Dr. Oz Show, and other media outlets. She is also a contributor to Women's Health magazine and has written several books, including The Smart Girl's Guide to Going Keto and The Smart Girl's Guide to Nutrition.</s>


In [ ]:
from transformers import TextStreamer

def stream(user_prompt):
    runtimeFlag = "cuda:0"

    inputs = tokenizer(user_prompt, return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=False)

    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=200, do_sample=True,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)

stream("what is the fastest way to improve my vertical jump?")
stream("what is the fastest way to improve my vertical jump?")

In [5]:
from transformers import TextStreamer

def stream(user_prompt):
    runtimeFlag = "cuda:0"
    # system_prompt = 'The conversation between Human and AI named PaulGPT\n'
    B_INST, E_INST = "[INST]", "[/INST]"

    # prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}\n{E_INST}"
    prompt = f"{B_INST} {user_prompt.strip()} {E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=False)

    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=200, do_sample=True,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)

stream("what is the fastest way to improve my vertical jump?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] what is the fastest way to improve my vertical jump? [/INST]

Squat, squat, squat. Squatting is a key component in improving vertical jump because it strengthens the fast twitch muscles and fast twitch fibers, which is what vertical jump is about. This is different from improving in a weight room because the purpose of squatting is not to increase vertical jump, but to strengthen the fast twitch muscles and fast twitch fibers, which is the key to improving vertical jump. This method may not be the fastest way to improve vertical jump, but it is definitely the best way to do so. For more information, check out the YouTube video link in the description. I will also include a link to a study that supports this method.

The study conducted by the Iowa State University research team found that a group of college basketball players who followed a 10-week program that focused on heavy squatting improved their vertical jump by an average of 1.8 inches. This is a significant improvem

In [ ]:
test_set = ["how can I jump higher","how to dunk?","are barefoot shoes actually help?","what are the most important factors to becoming an elite athlete?",
"what are the core principles behind nutrition?",
"how do i lose weight while gaining muscle",
"how do i get more energy?",
"are plant based diets beneficial for better health?",
"what are some quick wins to becoming a better athlete?",
"does contrast therapy (ice bath and hot sauna) actually work?",
"how exercises should I focus on at the gym to improve my basketball performance?"]

for i, prompt in enumerate(test_set):
    print(f"test case: {i+1}/{len(test_set)}")
    stream(prompt)
    print()